# Import Modules

In [1]:
import glob
import os
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import multiprocessing
import json
import time

import pandas as pd
from functools import partial
from tqdm.notebook import trange, tqdm
from sklearn.decomposition import PCA
# import umap
# import umap.plot 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader, TensorDataset, Subset
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import sys
# sys.path.insert(0, os.path.join(os.path.expanduser('~/Research/MyRepos/'),'SensoryMotorPred'))
# from datasets import WCDataset, WCShotgunDataset, WC3dDataset
import PreCNet as PN
from datasets import AudioDataset

import plotly.express as px
import plotly.graph_objects as go

import matplotlib as mpl
mpl.rcParams.update({'font.size':         24,
                     'axes.linewidth':    3,
                     'xtick.major.size':  5,
                     'xtick.major.width': 2,
                     'ytick.major.size':  5,
                     'ytick.major.width': 2,
                     'axes.spines.right': False,
                     'axes.spines.top':   False,
                     'font.sans-serif':  "Arial",
                     'font.family':      "sans-serif",
                    })

########## Checks if path exists, if not then creates directory ##########
def check_path(basepath, path):
    if path in basepath:
        return basepath
    elif not os.path.exists(os.path.join(basepath, path)):
        os.makedirs(os.path.join(basepath, path))
        print('Added Directory:'+ os.path.join(basepath, path))
        return os.path.join(basepath, path)
    else:
        return os.path.join(basepath, path)

rootdir = os.path.expanduser('~/Desktop/Research/Murray/git/PredAudio/')

# Set up partial functions for directory managing
join = partial(os.path.join,rootdir)
checkDir = partial(check_path,rootdir)
FigurePath = checkDir('Figures')

savefigs=False

# %matplotlib widget

2023-12-13 10:34:37.165444: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Test Loading

In [7]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Trial    = 7
TimeSize = 7

output_mode = 'error'
save_path   = os.path.expanduser('~/Desktop/Research/Murray/git/PredAudio/results/Tau01/NotStateful')
fileList    = sorted(glob.glob(os.path.join(save_path, 'PreCNetGRU_T{:03d}_N{:02d}_E*_Tau01_Visual_Netparams.json'.format(TimeSize,Trial))))
filename    = fileList[0]
Netpath = sorted(glob.glob(os.path.join(save_path,'PreCNetGRU_T{:03d}_N{:02d}_E*_Tau01_Visual*.pt'.format(TimeSize,Trial))))
print(filename)
if len(fileList)>0 and os.path.exists(fileList[0]):
    with open(fileList[0], 'r') as fp:
        netparams = json.load(fp)

netparams['Train_paths'] = os.path.expanduser('~/Desktop/Research/Murray/data/PredAudioData/soundFiles/Specs_train.npy')
netparams['Test_paths'] = os.path.expanduser('~/Desktop/Research/Murray/data/PredAudioData/soundFiles/Specs_test.npy')
netparams['save_path'] = save_path
netparams

/Users/Matt/Desktop/Research/Murray/git/PredAudio/results/Tau01/NotStateful/PreCNetGRU_T007_N07_E1500_Tau01_Visual_Netparams.json


{'Ahat_filt_sizes': [3, 3, 3, 3],
 'BatchSize': 64,
 'FiltNum': 4,
 'ImageSize': 2048,
 'KSize': 3,
 'LogDir': '/home/seuss/Research/PredAudio/results/Tau01/NotStateful/Logs/Trial_07',
 'Nepochs': 1500,
 'Overlap': 8,
 'R_filt_sizes': [3, 3, 3, 3],
 'R_stack_sizes': [4, 16, 64, 256],
 'Save_Grad': 0,
 'Stateful': 0,
 'Tau': 1,
 'Test_paths': '/Users/Matt/Desktop/Research/Murray/git/PredAudio/Specs_test.npy',
 'TimeSize': 7,
 'Train_paths': '/Users/Matt/Desktop/Research/Murray/git/PredAudio/Specs_train.npy',
 'Trial': 7,
 'WindSize': 16,
 'data_format': 'channels_first',
 'filename': 'PreCNetGRU_T007_N07_E1500_Tau01_Visual',
 'height': 128,
 'input_shape': [64, 7, 128, 16, 1],
 'layer_loss_weightsMode': 'L_0',
 'log_freq': 10,
 'lr': 0.005,
 'output_mode': 'error',
 'save_path': '/Users/Matt/Desktop/Research/Murray/git/PredAudio/results/Tau01/NotStateful',
 'stack_sizes': [1, 4, 16, 64],
 'use_motor': 0,
 'width': 16}

In [15]:
if netparams['data_format'] == 'channels_first':
    input_shape = (netparams['BatchSize'], netparams['TimeSize'], 1, netparams['height'], netparams['width'])
else:
    input_shape = (netparams['BatchSize'], netparams['TimeSize'], netparams['height'], netparams['width'], 1)

precnet,netparams = PN.getNetwork(netparams)
optimizer = torch.optim.Adam(precnet.parameters(), lr = netparams['lr'])
params = torch.load(Netpath[0], map_location=torch.device('cpu'))
precnet.load_state_dict(params['Model_state_dict'])
precnet.to(device)
print('Loaded Network')

Loaded Network


netparams['WindSize'] = 16
netparams['Overlap'] = netparams['WindSize']
netparams['TimeSize'] = 64//netparams['WindSize']
netparams['WindSize'],netparams['Overlap'],netparams['TimeSize'],

netparams['Overlap'] = netparams['WindSize']//2
netparams['Test_paths'] = '/home/seuss/Research/PredAudio/Specs_test_mfcc.npy'
netparams['Train_paths'] = '/home/seuss/Research/PredAudio/Specs_train_mfcc.npy'

In [11]:
tr_loss = 0.0
sum_trainLoss_in_epoch = 0.0
min_trainLoss_in_epoch = float('inf')
# tonespath = os.path.join(rootdir,'Specs_puretones_logscale.npy')
tonespath = os.path.join(rootdir,'Specs_mfcc.npy')

########## Create Datasets and DataLoaders ##########
Dataset_Train = AudioDataset(netparams['Train_paths'],netparams['WindSize'],netparams['Overlap'])
Dataset_Test = AudioDataset(netparams['Test_paths'],netparams['WindSize'],netparams['Overlap'])
Dataset_Pure = AudioDataset(tonespath,netparams['WindSize'],netparams['Overlap'])
num_workers = multiprocessing.cpu_count()//2
DataLoader_Train = DataLoader(Dataset_Train, batch_size=netparams['BatchSize'], shuffle=False, drop_last=True, num_workers=num_workers, pin_memory=True)
DataLoader_Test = DataLoader(Dataset_Test, batch_size=netparams['BatchSize'], shuffle=False, drop_last=True, num_workers=num_workers, pin_memory=True)
DataLoader_Pure = DataLoader(Dataset_Pure, batch_size=netparams['BatchSize'], shuffle=False, drop_last=True, num_workers=num_workers, pin_memory=True)

lr_maker  = lr_scheduler.StepLR(optimizer = optimizer, step_size = 100, gamma = 0.1)  # decay the lr every 50 epochs by a factor of 0.1
totsteps = 0

Epoch = 0
tr_loss = 0.0
sum_trainLoss_in_epoch = 0.0
min_trainLoss_in_epoch = float('inf')

initial_states = precnet.get_initial_states(input_shape)
states = initial_states
# step = 0

# precnet, optimizer = amp.initialize(precnet, optimizer, opt_level='O2')

AssertionError: Torch not compiled with CUDA enabled

## Grab States

In [ ]:
initial_states = precnet.get_initial_states(input_shape)
states = initial_states
h_tr = {'h{:d}'.format(n):[] for n in range(len(netparams['R_stack_sizes']))}
with torch.no_grad():
    for batch in tqdm(DataLoader_Train):
        output, states, frame_pred = precnet.grab_states(batch.to(device), states, grab_frame=True)
        for n, state in enumerate(states[:len(netparams['R_stack_sizes'])]):
            h_tr['h{:d}'.format(n)].append(output['h{:d}'.format(n)])
        initial_states = precnet.get_initial_states(input_shape)
        states = initial_states
for n in range(len(netparams['R_stack_sizes'])):
     h_tr['h{:d}'.format(n)] = np.stack(h_tr['h{:d}'.format(n)]).transpose(0,2,1,3,4,5).reshape((-1,netparams['TimeSize'],) + h_tr['h{:d}'.format(n)][0].shape[-3:])
        
# initial_states = precnet.get_initial_states(input_shape)
# states = initial_states
# h_te = {'h{:d}'.format(n):[] for n in range(len(netparams['R_stack_sizes']))}
# with torch.no_grad():
#     for batch in tqdm(DataLoader_Test):
#         output, states, frame_pred = precnet.grab_states(batch.to(device), states, grab_frame=True)
#         for n, state in enumerate(states[:len(netparams['R_stack_sizes'])]):
#             h_te['h{:d}'.format(n)].append(output['h{:d}'.format(n)])
#         initial_states = precnet.get_initial_states(input_shape)
#         states = initial_states
# for n in range(len(netparams['R_stack_sizes'])):
#      h_te['h{:d}'.format(n)] = np.stack(h_te['h{:d}'.format(n)]).transpose(0,2,1,3,4,5).reshape((-1,netparams['TimeSize'],) + h_te['h{:d}'.format(n)][0].shape[-3:])

        
# initial_states = precnet.get_initial_states(input_shape)
# states = initial_states
# h_pure = {'h{:d}'.format(n):[] for n in range(len(netparams['R_stack_sizes']))}
# with torch.no_grad():
#     for batch in tqdm(DataLoader_Pure):
#         output, states, frame_pred = precnet.grab_states(batch.to(device), states, grab_frame=True)
#         for n, state in enumerate(states[:len(netparams['R_stack_sizes'])]):
#             h_pure['h{:d}'.format(n)].append(output['h{:d}'.format(n)])
#         initial_states = precnet.get_initial_states(input_shape)
#         states = initial_states
# for n in range(len(netparams['R_stack_sizes'])):
#      h_pure['h{:d}'.format(n)] = np.stack(h_pure['h{:d}'.format(n)]).transpose(0,2,1,3,4,5).reshape((-1,netparams['TimeSize'],) + h_pure['h{:d}'.format(n)][0].shape[-3:])

In [12]:
tot_train = len(DataLoader_Train)*netparams['BatchSize']
tot_test = len(DataLoader_Test)*netparams['BatchSize']
tot_pure = len(DataLoader_Pure)*netparams['BatchSize']

labels_train = pd.read_csv(os.path.join(rootdir,'labels_train.csv'))
labels_train = labels_train.iloc[:tot_train]
labels_test = pd.read_csv(os.path.join(rootdir,'labels_test.csv'))
labels_test = labels_test.iloc[:tot_test]
labels_pure_df = pd.read_csv(os.path.join(rootdir,'labels_puretones_logscale.csv'))
labels_pure_df = labels_pure_df.iloc[:tot_pure]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/Matt/Desktop/Research/Murray/git/PredAudio/labels_puretones_logscale.csv'

In [ ]:
# output, states, frame_pred = precnet(batch.to(device), states, grab_frame=True)
FM_Pred = np.stack(frame_pred).transpose(1,0,2).reshape(netparams['BatchSize'], netparams['TimeSize'], netparams['height'], netparams['width'])

FM_Actual = batch.squeeze().numpy()
GT = np.squeeze(batch.detach().numpy())

In [ ]:
fmGT.shape

In [ ]:
os.path.join(FigurePath,'GT_Pred.png')

In [ ]:
7,12

add labels to image

In [13]:
n=7
plt.rcParams['figure.facecolor'] = 'white'

savefigs=False
GT = np.squeeze(batch.detach().numpy())
fmGT = torch.FloatTensor(GT[n]).unsqueeze(1)
fmP = torch.FloatTensor(FM_Pred[n]).unsqueeze(1)
GT_Grid = torchvision.utils.make_grid(fmGT[:,:,:,8:],nrow=netparams['TimeSize'])
P_Grid = torchvision.utils.make_grid(fmP[:,:,:,8:],nrow=netparams['TimeSize'])


fig1, axs = plt.subplots(2, 1, figsize = (20,10))
axs[0].imshow(GT_Grid[0], aspect='auto',  origin='lower') #cmap='gray'
axs[0].axis('off')
axs[0].set_title('Ground Truth, Digit: {:d}'.format(labels_train['digits'][n]))
axs[1].imshow(P_Grid[0],  aspect='auto',  origin='lower') #cmap='gray'
axs[1].axis('off')
axs[1].set_title('Prediction')
plt.tight_layout()
plt.show()
if savefigs:
    fig1.savefig(os.path.join(FigurePath,'GT_Pred_{:d}.pdf'.format(n)), facecolor=fig1.get_facecolor(), transparent=True)


NameError: name 'batch' is not defined

In [ ]:
for key in h_tr.keys():
    print(h_tr[key].shape, h_tr[key].shape[-1]*h_tr[key].shape[-2]*h_tr[key].shape[-3])

In [ ]:
8*128*32, 64*64*4

## PCA

In [14]:
N_comp=200
layer = 2
pcaL = PCA(n_components=N_comp)
zcomps = pcaL.fit_transform(h_tr['h{:d}'.format(layer)][:,-1].reshape(tot_train,-1))

exp_var_cumul = np.cumsum(pcaL.explained_variance_ratio_)
fig = px.area(x=range(1, exp_var_cumul.shape[0] + 1),
        y=exp_var_cumul,
        labels={"x": "# Components", "y": "Explained Variance"},
        title='ncomps 95%: {}/{}'.format(len(exp_var_cumul[exp_var_cumul<.95]),N_comp)
        )
fig.show()

N_comp=200
layer = 2
pcaL = PCA(n_components=N_comp)
zcomps = pcaL.fit_transform(h_te['h{:d}'.format(layer)][:,-1].reshape(tot_train,-1))

exp_var_cumul = np.cumsum(pcaL.explained_variance_ratio_)
fig = px.area(x=range(1, exp_var_cumul.shape[0] + 1),
        y=exp_var_cumul,
        labels={"x": "# Components", "y": "Explained Variance"},
        title='ncomps 95%: {}/{}'.format(len(exp_var_cumul[exp_var_cumul<.95]),N_comp)
        )
fig.show()

N_comp=200
layer = 2
pcaL = PCA(n_components=N_comp)
zcomps = pcaL.fit_transform(h_pure['h{:d}'.format(layer)][:,-1].reshape(tot_pure,-1))

exp_var_cumul = np.cumsum(pcaL.explained_variance_ratio_)
fig = px.area(x=range(1, exp_var_cumul.shape[0] + 1),
        y=exp_var_cumul,
        labels={"x": "# Components", "y": "Explained Variance"},
        title='ncomps 95%: {}/{}'.format(len(exp_var_cumul[exp_var_cumul<.95]),N_comp)
        )
fig.show()

NameError: name 'h_tr' is not defined

In [ ]:
zcomps.shape

In [ ]:

def PCA_States(ht,netparams,labels_df,tot_trials,FigPath,N_comp=30,savefigs=False,Type='Train'):
    ztot = []
    for layer in trange(len(netparams['R_stack_sizes'])):
        pcaL = PCA(n_components=N_comp)
        zcomps = pcaL.fit_transform(ht['h{:d}'.format(layer)][:,:].reshape(tot_trials,-1))
        ztot.append(zcomps)
        exp_var_cumul = np.cumsum(pcaL.explained_variance_ratio_)
        fig = px.area(x=range(1, exp_var_cumul.shape[0] + 1),
                y=exp_var_cumul,
                labels={"x": "# Components", "y": "Explained Variance"},
                title='ncomps 95%: {}/{}'.format(len(exp_var_cumul[exp_var_cumul<.95]),N_comp)
                )
    #     fig.show()
        if savefigs:

            labels = {
                str(i): f"PC {i+1} ({var:.1f}%)"
                for i, var in enumerate(pcaL.explained_variance_ratio_ * 100)
            }

            fig2 = px.scatter_matrix(
                zcomps,
                labels=labels,
                dimensions=range(2),
                color=labels_df['digits'],

            )
            fig2.update_traces(diagonal_visible=False)
            fig2.update_layout(autosize=False,
                              coloraxis_colorbar=dict(title='Time'),
                              width=1000,
                              height=1000,
                              )
            fig.write_image(os.path.join(check_path(FigPath,Type),'ExplainedVar_L{:d}_Trial{}.png'.format(layer,Trial)))
            fig2.write_image(os.path.join(check_path(FigPath,Type),'PCAMat_L{:d}_Trial{}.png'.format(layer,Trial)))
    return ztot

In [ ]:
N_comp=9
FigPath = check_path(FigurePath,'{}'.format(netparams['filename']))
savefigs = False
ztot_tr = PCA_States(h_tr,netparams,labels_train,tot_train,FigPath,N_comp=N_comp,savefigs=True,Type='Train_LogScale')
ztot_te = PCA_States(h_te,netparams,labels_test,tot_test,FigPath,N_comp=N_comp,savefigs=True,Type='Test_LogScale')
ztot_pure = PCA_States(h_pure,netparams,labels_pure_df,tot_pure,FigPath,N_comp=N_comp,savefigs=True,Type='Pure_LogScale')

In [ ]:
layer=0
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pcaL.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    ztot_tr[layer],
    labels=labels_test,
    dimensions=range(4),
    color=labels_train['digits'],
   
)
fig.update_traces(diagonal_visible=False)
fig.update_layout(autosize=False,
                  coloraxis_colorbar=dict(title='Time'),
                  width=1000,
                  height=1000,
                  )
fig.show()

In [ ]:
layer=3
fig = go.Figure(data=[go.Scatter3d(x=ztot_tr[layer][:,0], y=ztot_tr[layer][:,1], z=ztot_tr[layer][:,2],
                                   mode='markers',
                                   marker=dict(
                                                size=5,
                                                color= pd.factorize(labels_train['speaker'])[0], # labels_train['digits'], # pd.factorize(labels_test['speaker'])[0], #
                                                colorbar=dict(title='Digit'),
                                                opacity=0.8),
#                                    line=dict(
#                                              color=np.arange(zcomps[t:t+dt,0].shape[0])/30,
#                                              width=2)
                                   )])
fig.update_layout(autosize=False,
                  coloraxis_colorbar=dict(title='Time (s)'),
                  width=500,
                  height=500,
                  )
fig.show()

## Linear Regression Testing

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

In [ ]:
# layer = 3
data_type = 'PCA'
def train_classifer(data,ht,labels,netparams,tot_trials,data_type=data_type):
    scores = []
    for layer in trange(len(netparams['R_stack_sizes'])):
        if data_type == 'PCA':
            X = data[layer]
        else:
            X = ht['h{:d}'.format(layer)][:,1].reshape(tot_trials,-1) #  

        clf = make_pipeline(StandardScaler(),
#                             LinearSVC(random_state=0, tol=1e-5, max_iter=10000),)
                            LogisticRegression(random_state=0, tol=1e-5, max_iter=10000,n_jobs=-1)) # 

        # Split data into 50% train and 50% test subsets
        X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.5, shuffle=False, random_state=1)

        # Learn the digits on the train subset
        clf.fit(X_train, y_train)

        predicted = clf.predict(X_test)
        scores.append(metrics.accuracy_score(y_test, predicted))
        print('Layer: {}, Score: {}'.format(layer, scores[layer]))
#         print(f"Classification report for classifier layer{layer} {clf}:\n"
#               f"{metrics.classification_report(y_test, predicted)}\n")
    return scores


In [ ]:
scores_tr = train_classifer(ztot_tr,h_tr,labels_train['digits'].to_numpy(),netparams,tot_train,data_type='PCA')
scores_te = train_classifer(ztot_te,h_te,labels_test['digits'].to_numpy(),netparams,tot_test,data_type='PCA')
scores_pure = train_classifer(ztot_pure,h_pure,labels_pure_df['digits'].to_numpy(),netparams,tot_pure,data_type='PCA')


In [ ]:
ztot_tr2 = np.hstack(ztot_tr)
ztot_te2 = np.hstack(ztot_te)
ztot_pure2 = np.hstack(ztot_pure)

In [ ]:
# layer = 3
data_type = 'PCA'
def train_classifer_compiled(X,ht,labels,netparams,tot_trials,data_type=data_type):
    scores = []
    clf = make_pipeline(StandardScaler(),
#                             LinearSVC(random_state=0, tol=1e-5, max_iter=10000),)
                        LogisticRegression(random_state=0, tol=1e-5, max_iter=10000,n_jobs=-1)) # 

    # Split data into 50% train and 50% test subsets
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, shuffle=False, random_state=1)

    # Learn the digits on the train subset
    clf.fit(X_train, y_train)

    predicted = clf.predict(X_test)
    scores.append(metrics.accuracy_score(y_test, predicted))
    print('Layer: all, Score: {}'.format(scores))
#         print(f"Classification report for classifier layer{layer} {clf}:\n"
#               f"{metrics.classification_report(y_test, predicted)}\n")
    return scores


In [ ]:
scores_tr2 = train_classifer_compiled(ztot_tr2,h_tr,labels_train['digits'].to_numpy(),netparams,tot_train,data_type='PCA')
scores_te2 = train_classifer_compiled(ztot_te2,h_te,labels_test['digits'].to_numpy(),netparams,tot_test,data_type='PCA')
scores_pure2 = train_classifer_compiled(ztot_pure2,h_pure,labels_pure_df['digits'].to_numpy(),netparams,tot_pure,data_type='PCA')


In [ ]:
scores_tr = np.array(scores_tr)
scores_te = np.array(scores_te)
scores_pure = np.array(scores_pure)

# Create Grant Figures

In [ ]:
N_comp=7
FigPath = check_path(FigurePath,'{}'.format(netparams['filename']))

for N_comp in range(1,20):
    ztot_tr = PCA_States(h_tr,netparams,labels_train,tot_train,FigPath,N_comp=N_comp,savefigs=False,Type='Train_LogScale')
    ztot_te = PCA_States(h_te,netparams,labels_test,tot_test,FigPath,N_comp=N_comp,savefigs=False,Type='Test_LogScale')
    ztot_pure = PCA_States(h_pure,netparams,labels_pure_df,tot_pure,FigPath,N_comp=N_comp,savefigs=False,Type='Pure_LogScale')

    scores_tr = train_classifer(ztot_tr,h_tr,labels_train['digits'].to_numpy(),netparams,tot_train,data_type='PCA')
    scores_te = train_classifer(ztot_te,h_te,labels_test['digits'].to_numpy(),netparams,tot_test,data_type='PCA')
    scores_pure = train_classifer(ztot_pure,h_pure,labels_pure_df['digits'].to_numpy(),netparams,tot_pure,data_type='PCA')
    scores_tr = np.array(scores_tr)
    scores_te = np.array(scores_te)
    scores_pure = np.array(scores_pure)

    ztot_tr2 = np.hstack(ztot_tr)
    ztot_te2 = np.hstack(ztot_te)
    ztot_pure2 = np.hstack(ztot_pure)

    scores_tr2 = train_classifer_compiled(ztot_tr2,h_tr,labels_train['digits'].to_numpy(),netparams,tot_train,data_type='PCA')
    scores_te2 = train_classifer_compiled(ztot_te2,h_te,labels_test['digits'].to_numpy(),netparams,tot_test,data_type='PCA')
    scores_pure2 = train_classifer_compiled(ztot_pure2,h_pure,labels_pure_df['digits'].to_numpy(),netparams,tot_pure,data_type='PCA')


    fig, ax = plt.subplots(1,figsize=(12,5))
    ax.bar([1,2],scores_te[[0,-1]],color='b',label='Spoken Digits')
    ax.bar([4,5],scores_pure[[0,-1]],color='g', label='Pure Tones')
    ax.axhline(y=1/len(np.unique(labels_train['digits'])),linestyle='--', color='red')
    ax.bar(7,scores_te2,color='b',)
    ax.bar(8,scores_pure2,color='g',)
    ax.set_ylim(0,1.1)
    ax.legend(bbox_to_anchor=(1.01, 1))
    ax.set_xlabel('Layer #')
    ax.set_ylabel('Accuracy')
    ax.set_xticks(np.arange(9))
    ax.set_xticklabels(['','1','4','','1','4','','all','all'])
    # ax.set_title('Labels=Digits')
    plt.tight_layout()
#     plt.show()
    fig.savefig(os.path.join(FigPath,'LogisticReg_Accuracy_{}_LogScale_PCA{:d}_withALL.pdf'.format(data_type,N_comp)))


    fig, ax = plt.subplots(1,figsize=(8,5))
    ax.bar([1,2],scores_te[[0,-1]],color='b',label='Spoken Digits')
    ax.bar([4,5],scores_pure[[0,-1]],color='g', label='Pure Tones')
    ax.axhline(y=1/len(np.unique(labels_train['digits'])),linestyle='--', color='red')
    # ax.set_ylim(0,1.1)
    ax.legend()#,bbox_to_anchor=(1.01, 1))
    ax.set_xlabel('Layer #')
    ax.set_ylabel('Accuracy')
    ax.set_xticks(np.arange(6))
    ax.set_xticklabels(['','1','4','','1','4'])
    # ax.set_title('Labels=Digits')
    plt.tight_layout()
#     plt.show()

    fig.savefig(os.path.join(FigPath,'LogisticReg_Accuracy_{}_LogScale_PCA{:d}.pdf'.format(data_type,N_comp)))


    fig, ax = plt.subplots(1,figsize=(10,5))
    ax.plot(scores_tr, 'b.-',markersize=20, linewidth=3)
    ax.plot(scores_te, 'r.-',markersize=20, linewidth=3)
    ax.plot(scores_pure, 'g.-',markersize=20, linewidth=3)
    ax.axhline(y=1/len(np.unique(labels_train['digits'])),linestyle='--', color='red')
    ax.plot(4,scores_tr2, 'b.-',markersize=20, linewidth=3)
    ax.plot(4,scores_te2, 'r.-',markersize=20, linewidth=3)
    ax.plot(4,scores_pure2, 'g.-',markersize=20, linewidth=3)


    ax.set_ylim(0,1.1)
    ax.legend(['Train','Test','Pure Tones','Random'],bbox_to_anchor=(1.01, 1))
    ax.set_xlabel('Layer #')
    ax.set_ylabel('Accuracy')
    ax.set_xticks(np.arange(5))
    ax.set_xticklabels(['0','1','2','3','all'])
    ax.set_title('Labels=Digits')
    plt.tight_layout()
#     plt.show()
    fig.savefig(os.path.join(FigPath,'LogisticReg_Accuracy_{}_LogScale_PCA{:d}_Scatter.pdf'.format(data_type,N_comp)))


Some difference in decoding accuracy across layers and is different for different type of stimuli. 

Pure tone input to trained PreCnet, want decoding in first layer to be the highest. 

In [ ]:
ztot_pure.shape

In [ ]:
fig = plt.figure(figsize=(20,20))
disp = metrics.plot_confusion_matrix(clf, X_test, y_test)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

In [ ]:
clf = LogisticRegressionCV(cv=2, random_state=0, n_jobs=-1).fit(X, y)
# clf.predict(X[:2, :])

# clf.predict_proba(X[:2, :]).shape

# clf.score(X, y)

# Pure Tones

In [ ]:
import torchvision.transforms as TV
import torchaudio.transforms as AV

In [ ]:
import librosa

def plot_spectrogram(spec, title=None, ylabel='freq_bin', aspect='auto', xmax=None):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or 'Spectrogram (db)')
    axs.set_ylabel(ylabel)
    axs.set_xlabel('frame')
    im = axs.imshow(librosa.power_to_db(spec), origin='lower', aspect=aspect)
    if xmax:
        axs.set_xlim((0, xmax))
    fig.colorbar(im, ax=axs)
    plt.show(block=False)

In [ ]:
np.logspace(1,4,10,base=10)

In [ ]:


sig=0
f = 1
phase = 0
ntones = 10
toneslist = [n*1000 for n in range(1,ntones+1)]
ntrials = 64
nsamples = 9000
times = np.linspace(0,2,nsamples)
tones = np.zeros((ntrials,ntones,nsamples))
labels_pure = np.zeros((ntrials,ntones,1))
amod = 5*np.random.rand(ntrials,ntones)

freq, trials, = [], []
for trial in trange(ntrials):
    for n, f in enumerate(toneslist):
        tones[trial,n] = amod[trial,n]*np.sin(f/(2*np.pi)*times + phase) + np.random.normal(loc=0,scale=sig,size=(nsamples))
        freq.append(f)
        trials.append(trial)
labels_pure_df = pd.DataFrame({'digits':freq,'Trial':trials})


In [ ]:
tones.shape

In [ ]:
plt.plot(times[:100],tones[0,9,:100])
plt.show()

In [ ]:
n_fft = 256
win_length = None
hop_length = 128

# define transformation
spects = AV.Spectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
#     center=True,
#     pad_mode="reflect",
#     power=2.0,
)
# Perform transformation
spec = spects(torch.tensor(tones[0,-1],dtype=torch.float32))

# print_stats(spec)
plot_spectrogram(spec, title='torchaudio')
print(spec.shape)

In [ ]:
nf,nt = (128,71)
resize = TV.Resize((nf,nt))
tones2 = tones.reshape(-1,nsamples)
data_spec1 = np.zeros((tones2.shape[0],1,nf,nt))
for n in range(tones2.shape[0]):
    mfcc = spects(torch.tensor(tones2[n],dtype=torch.float32)).numpy() #mfcc_transform
#     mfcc = mfcc_transform(torch.tensor(data1[n],dtype=torch.float32)).numpy() #mfcc_transform
    spec2 = resize(torch.Tensor(mfcc).unsqueeze(0))
    data_spec1[n,:] = librosa.power_to_db(spec2.numpy()) 


In [ ]:
np.save(os.path.join(rootdir,'Specs_puretones_logscale_nonoise.npy'),data_spec1)
labels_pure_df.to_csv(os.path.join(rootdir,'labels_puretones_logscale_nonoise.csv'))

## Grab States for Pure Tones

In [ ]:
labels_pure_df = pd.read_csv(os.path.join(rootdir,'labels_puretones.csv'))
initial_states = precnet.get_initial_states(input_shape)
states = initial_states
ht_pure = {'h{:d}'.format(n):[] for n in range(len(netparams['R_stack_sizes']))}
with torch.no_grad():
    for batch in tqdm(DataLoader_Pure):
        output, states, frame_pred = precnet.grab_states(batch.to(device), states, grab_frame=True)
        for n, state in enumerate(states[:len(netparams['R_stack_sizes'])]):
            ht_pure['h{:d}'.format(n)].append(output['h{:d}'.format(n)])
        initial_states = precnet.get_initial_states(input_shape)
        states = initial_states
for n in range(len(netparams['R_stack_sizes'])):
     ht_pure['h{:d}'.format(n)] = np.stack(ht_pure['h{:d}'.format(n)]).transpose(0,2,1,3,4,5).reshape((-1,netparams['TimeSize'],) + ht_pure['h{:d}'.format(n)][0].shape[-3:])

In [ ]:
FigPath

In [ ]:
tot_pure = len(Dataset_Pure)
N_comp=200
FigPath = check_path(FigurePath,'{}'.format(netparams['filename']))
savefigs = True
ztot = []
for layer in trange(len(netparams['R_stack_sizes'])):
    pcaL = PCA(n_components=N_comp)
    zcomps = pcaL.fit_transform(ht['h{:d}'.format(layer)][:,:].reshape(tot_pure,-1))
    ztot.append(zcomps)
    exp_var_cumul = np.cumsum(pcaL.explained_variance_ratio_)
    fig = px.area(x=range(1, exp_var_cumul.shape[0] + 1),
            y=exp_var_cumul,
            labels={"x": "# Components", "y": "Explained Variance"},
            title='ncomps 95%: {}/{}'.format(len(exp_var_cumul[exp_var_cumul<.95]),N_comp)
            )
#     fig.show()
    
    labels = {
        str(i): f"PC {i+1} ({var:.1f}%)"
        for i, var in enumerate(pcaL.explained_variance_ratio_ * 100)
    }

    fig2 = px.scatter_matrix(
        zcomps,
        labels=labels,
        dimensions=range(4),
        color=labels_pure_df['Freq'],

    )
    fig2.update_traces(diagonal_visible=False)
    fig2.update_layout(autosize=False,
                      coloraxis_colorbar=dict(title='Time'),
                      width=1000,
                      height=1000,
                      )
    if savefigs:
        fig.write_image(os.path.join(FigPath,'ExplainedVar_L{:d}_Trial{}_PureTones.png'.format(layer,Trial)))
        fig2.write_image(os.path.join(FigPath,'PCAMat_L{:d}_Trial{}_PureTones.png'.format(layer,Trial)))


In [ ]:
# layer = 3
scores = []
for layer in trange(len(netparams['R_stack_sizes'])):
    X = ztot[layer]#  ht['h{:d}'.format(layer)][:,1].reshape(tot_pure,-1) # 
#     y = labels_train['digits'].to_numpy()

    # clf = svm.LinearSVC(max_iter=5000)
    clf = make_pipeline(StandardScaler(),
                        LinearSVC(random_state=0, tol=1e-5, max_iter=10000)) # 

    labels = labels_pure_df['Freq'].to_numpy()# pd.factorize(labels_train['speaker'])[0] # 
    # Split data into 50% train and 50% test subsets
    X_train, X_test, y_train, y_test = train_test_split(
        X, labels, test_size=0.5, shuffle=False, random_state=1)

    # Learn the digits on the train subset
    clf.fit(X_train, y_train)

    predicted = clf.predict(X_test)
    scores.append(metrics.accuracy_score(y_test, predicted))

    print(f"Classification report for classifier layer{layer} {clf}:\n"
          f"{metrics.classification_report(y_test, predicted)}\n")


In [ ]:
fig, ax = plt.subplots(1,figsize=(8,5))
ax.plot(scores, '.-',markersize=20, linewidth=3)
ax.axhline(y=1/len(np.unique(y_test)),linestyle='--', color='red')
ax.set_ylim(0,1.1)
ax.legend(['SVM','Random'],bbox_to_anchor=(1.01, 1))
ax.set_xlabel('Layer #')
ax.set_ylabel('Accuracy')
ax.set_title('Labels=Freq')
plt.tight_layout()
plt.show()
fig.savefig(os.path.join(FigPath,'SVM_Accuracy_PureTones.png'))

# Testing Plots

In [ ]:
batch_iter = iter(DataLoader_Train)
batch = batch_iter.next()


In [ ]:
output, states, frame_pred = precnet(batch.to(device), states, grab_frame=True)

In [ ]:
FM_Pred = np.stack(frame_pred).transpose(1,0,2).reshape(netparams['BatchSize'], netparams['TimeSize'], netparams['height'], netparams['width'])

FM_Actual = batch.squeeze().numpy()
GT = np.squeeze(batch.detach().numpy())

In [ ]:
GT.shape, FM_Pred.shape

In [ ]:
from numba import jit,njit, prange
import time

# Calculate the error between the prediction and actual fram separated by delta t
@njit(parallel=True)
def parallel_diff(FM_Pred,FM_Actual,diffrange=10):
    BatchSize,tlength,width,height = FM_Actual.shape
    # Assumes FM_Pred and FM_Actual have same dimensions.
    assert FM_Pred.shape == FM_Actual.shape,'Dimensions must be equal'
    # Diff in frame loss
    diffloss = np.zeros((tlength, diffrange*2+1))
    diffloss[:] = np.nan
    
    for fm in prange(tlength):# trange(0,tlength, desc='Frame Num',leave=False): #
        for ind, tau in enumerate(np.arange(-diffrange, diffrange+1)):
            if ((fm+tau) <= (tlength-tau)) & ((fm+tau)>=0):
                diffloss[fm,ind] = np.nanmean(np.abs(FM_Pred[:,fm,:,:] - FM_Actual[:,fm+tau,:,:]))
    return diffloss

In [ ]:
diffrange  = 5
start_time = time.time()
diffloss   = parallel_diff(FM_Pred.astype(float),GT.astype(float),diffrange=diffrange)
end_time   = time.time()
Tot_Time   = end_time - start_time
print(Tot_Time)

STD       = np.nanstd(diffloss,axis=0)
DiffAvg   = np.nanmean(diffloss,axis=0)


In [ ]:
fig, ax = plt.subplots(1,figsize=(5,5))
x=np.arange(-diffrange,diffrange+1)+1
ax.plot(x,DiffAvg, 'k', LineWidth=3 )
ax.fill_between(x,DiffAvg-STD,DiffAvg+STD,alpha=.5,color='k') 
plt.show()

In [ ]:
np.stack(frame_pred).shape

In [ ]:
# frame_pred = np.stack(frame_pred).transpose(1,0,2).reshape(netparams['BatchSize'], netparams['TimeSize'], netparams['height'], netparams['width'])
GT = np.squeeze(batch.detach().numpy())
fmGT = torch.FloatTensor(GT[0]).unsqueeze(1)
fmP = torch.FloatTensor(FM_Pred[0]).unsqueeze(1)
GT_Grid = torchvision.utils.make_grid(fmGT,nrow=netparams['TimeSize'])
P_Grid = torchvision.utils.make_grid(fmP,nrow=netparams['TimeSize'])

fig1, axs = plt.subplots(2, 1, figsize = (20,10))
axs[0].imshow(GT_Grid[0], aspect='auto',  origin='lower') #cmap='gray'
axs[1].imshow(P_Grid[0],  aspect='auto',  origin='lower') #cmap='gray'
axs[0].axis('off')
axs[1].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
np.mean(np.abs(GT[0,1:] - FM_Pred[0,1:]))

In [ ]:
plt.imshow(np.stack(FM_Pred[0,:]))

In [ ]:
BatchSize,tlength,width,height = FM_Actual.shape
# Assumes FM_Pred and FM_Actual have same dimensions.
assert FM_Pred.shape == FM_Actual.shape,'Dimensions must be equal'
# Diff in frame loss
diffloss = np.zeros((tlength, diffrange*2+1))
diffloss[:] = np.nan

for fm in trange(tlength):# trange(0,tlength, desc='Frame Num',leave=False): #
    for ind, tau in enumerate(np.arange(-diffrange, diffrange+1)):
        if ((fm+tau) <= (tlength-tau)) & ((fm+tau)>=0):
            diffloss[fm,ind] = np.nanmean(np.abs(FM_Pred[:,fm,:,:] - FM_Actual[:,fm+tau,:,:]))

In [ ]:
fm = 1
for ind, tau in enumerate(np.arange(-diffrange, diffrange+1)):
    if ((fm+tau) <= (tlength-tau)) & ((fm+tau)>=0):
        diffloss[fm,ind] = np.nanmean(np.abs(FM_Pred[:,fm,:,:] - FM_Actual[:,fm+tau,:,:]))

In [ ]:
tau = 0
((fm+tau) <= (tlength-tau)), ((fm+tau)>=0)

In [ ]:
np.nanmean(np.abs(FM_Pred[:,fm,:,:] - FM_Actual[:,fm+tau,:,:]))

In [ ]:
plt.plot(np.arange(-diffrange, diffrange+1)+1,np.nanmean(diffloss, axis=0))

In [ ]:
np.nanmean(diffloss, axis=0)

# Feature Visualization 

In [ ]:

import torchvision.transforms as T


In [ ]:
_ = model.to(device).eval()

In [ ]:
X = next(iter(train_dataloader))

In [ ]:
list(map(lambda x: x[0], precnet.hd.named_children()))


In [ ]:
# We will register a forward hook to get the output of the layers

activation = {} # to store the activation  of a layer
def create_hook(name):
    def hook(m, i, o):
        # copy the output of the given layer
        activation[name] = o
       
    return hook


# register a forward hook for layer inception4a i.e. the first inception layer
precnet.hd.register_forward_hook(create_hook('conv0'))

In [ ]:


normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


# undo the above normalization if and when the need arises 
denormalize = T.Normalize(mean = [-0.485/0.229],  #, -0.456/0.224, -0.406/0.225],
                                   std = [1/0.229])  #, 1/0.224, 1/0.225] )

def random_image(Height = 28, Width = 28, device = 'cpu', requires_grad=False, optimizer=None, lr = 0.01):
    img = np.single(np.random.uniform(0,1, (16, Height, Width))) # we need the pixel values to be of type float32
    im_tensor = torch.from_numpy(img).to(device).requires_grad_(requires_grad) # normalize the image to have requisite mean and std. dev.
    print("img_shape:{}, img_dtype: {}".format(im_tensor.shape, im_tensor.dtype ))
    
    if optimizer:
        if requires_grad:
            return im_tensor, optimizer([im_tensor], lr = lr)
        else: 
            print('Error: Optimizer cannot be used on an image without setting its requires_grad_  ')
    
    return im_tensor

In [ ]:

# function to massage img_tensor for using as input to plt.imshow()
def image_converter(im):
    
    # move the image to cpu
    im_copy = im.cpu()
    
    # for plt.imshow() the channel-dimension is the last
    # therefore use transpose to permute axes
    im_copy = im_copy.clone().detach().numpy().transpose(1,2,0)
    # clip negative values as plt.imshow() only accepts 
    # floating values in range [0,1] and integers in range [0,255]
    im_copy = im_copy.clip(0, 1) 
    
    return im_copy

In [ ]:
# class to compute image gradients in pytorch
class gradients(nn.Module):
    def __init__(self, weight):
        super().__init__()
        k_depth, k_height, k_width = weight.shape[2:]
        # assuming that the height and width of the kernel are always odd numbers
        padding_x = int((k_height-1)/2)
        padding_y = int((k_width-1)/2)
        padding_z = int((k_depth-1)/2)
        
        # convolutional layer with 2 output channels corresponding to the x and the y gradients
        self.conv = nn.Conv3d(1, 2, (k_depth,k_height, k_width), bias = False, 
                              padding = (padding_z, padding_x, padding_y) )
        # initialize the weights of the convolutional layer to be the one provided
        if self.conv.weight.shape == weight.shape:
            self.conv.weight = nn.Parameter(weight)
            self.conv.weight.requires_grad_(False)
        else:
            print('Error: The shape of the given weights is not correct')
    
    def forward(self, x):
        return self.conv(x)

In [ ]:
gradLayer = gradients(torch.from_numpy(grad_filters).unsqueeze(1).type(torch.FloatTensor))


In [ ]:
H = 64 # height of input image
W = 64 # width of input image
img_tensor, optimizer = random_image(Height = H, Width = W, device = device, 
                                     requires_grad = True, optimizer = optim.Adam, lr = 0.01)

In [ ]:
im_grid = torchvision.utils.make_grid(img_tensor.cpu().unsqueeze(1),nrow=4,normalize=True)
# # Starting Image
plt.imshow(im_grid.permute(1,2,0))                                                               
plt.axis('off')
# plt.title('starting image')
# plt.show()

In [ ]:
# understanding the shape of the output produced by a given layer
image = img_tensor.clone().detach().to(device).requires_grad_(True)
model(image.unsqueeze(0).unsqueeze(0))
activation['conv0'].shape

In [ ]:
model.eval()


num_epochs = 10000
display_every = 1000
unit_idx = 10 # unit of the convolution layer that we wish to visualize

for epoch in range(num_epochs):
    
    optimizer.zero_grad()
    model(img_tensor.unsqueeze(0).unsqueeze(0))
    layer_out = activation['conv0']
    loss = -layer_out[0, unit_idx].mean() 
    loss.backward()
    optimizer.step()
    # normalize the updated img_tensor to have pytorch specified mean and std. dev.
    # img_tensor = normalize(img_tensor.clone().detach()).requires_grad_(True)
    # the above step of renormalizing the updated img_tensor does not work: the activation remains frozen at ~ 2.368 and no patterns seem to develop in the image
    # Will have to understand this further  
    
    if epoch % display_every == 0:
        print('epoch: {}/{}, activation: {}'.format(epoch, num_epochs, -loss))
        im_grid = torchvision.utils.make_grid(img_tensor.cpu().unsqueeze(1),nrow=4,normalize=True)
        plt.imshow(im_grid.permute(1,2,0))
        plt.axis('off')
        plt.title('input image after {} epochs'.format(epoch))
        plt.show()

In [ ]:
'''
forward pass through the model to get the scores, note that VGG-19 model doesn't perform softmax at the end
and we also don't need softmax, we need scores, so that's perfect for us.
'''

scores = model(X.to(device))[0]

# Get the index corresponding to the maximum score and the maximum score itself.
score_max_index = scores.argmax(axis=1)

In [ ]:
score_max = scores[0,score_max_index]

'''
backward function on score_max performs the backward pass in the computation graph and calculates the gradient of 
score_max with respect to nodes in the computation graph
'''
score_max.backward()

'''
Saliency would be the gradient with respect to the input image now. But note that the input image has 3 channels,
R, G and B. To derive a single class saliency value for each pixel (i, j),  we take the maximum magnitude
across all colour channels.
'''
saliency, _ = torch.max(X.grad.data.abs(),dim=1)

# code to plot the saliency map as a heatmap
plt.imshow(saliency[0], cmap=plt.cm.hot)
plt.axis('off')
plt.show()

In [ ]:
del scores, score_max_index, X
torch.cuda.empty_cache()
gc.collect()

In [ ]:
score_max.shape

# Dataset Testing

In [ ]:
class AudioDataset(Dataset):
    def __init__(self, data_path, wind_size=32, transform=None):
        
        self.data = np.load(data_path)
        self.wind_size = wind_size
        self.transform = transform
        

    def __len__(self):
        return(self.data.shape[0])
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = np.zeros((self.data.shape[-1]//self.wind_size,self.data.shape[-2],self.wind_size))
        for n in range(self.data.shape[-1]//self.wind_size):
            sample[n] = self.data[idx,:,:,(n*self.wind_size):(n*self.wind_size+self.wind_size)]
        sample = (sample - np.min(sample))/(np.max(sample)-np.min(sample))
        sample = np.clip(sample, 2e-30, 1).astype(np.float32)

        sample = torch.Tensor(sample[:,np.newaxis])

        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [ ]:
dataset = AudioDataset(data_path_train,wind_size)
train_dataloader = DataLoader(dataset,
                              batch_size= 60,
                              shuffle = False,
                              drop_last=False,
#                               num_workers=7,
                              pin_memory=False,)

In [ ]:
batch = next(iter(train_dataloader))

In [ ]:
batch.shape

In [ ]:
batch_iter = iter(DataLoader_Train)
batch = batch_iter.next()

In [ ]:
batch.shape

In [ ]:
plt.imshow(np.hstack(batch[3,:,0]), aspect='auto', origin='lower')